In [1]:
from data_loader_df import *
import sys
import numpy
import pandas as pd

In [5]:
train_file_path = "../data/adult.data"
test_file_path = "../data/adult.test"
train_X, train_Y, val_X, val_Y, test_X, test_Y = load_all_data(train_file_path, test_file_path, valid_rate=0.1, is_df=False, norm=False, one_hot=False)


In [17]:
print train_X[0]
print val_X[0]
print test_X[0]
print train_X.shape, val_X.shape, test_X.shape

[62 ' Local-gov' 68268 ' HS-grad' 9 ' Married-civ-spouse'
 ' Transport-moving' ' Husband' ' White' ' Male' 0 0 40 ' United-States']
[57 ' Private' 96779 ' 5th-6th' 3 ' Married-civ-spouse' ' Craft-repair'
 ' Husband' ' White' ' Male' 0 0 40 ' United-States']
[25 ' Private' 226802 ' 11th' 7 ' Never-married' ' Machine-op-inspct'
 ' Own-child' ' Black' ' Male' 0 0 40 ' United-States']
(27158, 14) (3004, 14) (15060, 14)


In [23]:
from sklearn.preprocessing import LabelEncoder
#For trees, we should use label encoding rather than one hot to transfrom category feature
#preprocess: combine all the data to do label encoding
dataset = np.concatenate((train_X, val_X), axis=0)
dataset = np.concatenate((dataset, test_X), axis=0)
#trasfrom all features if needed
for j in xrange(14):#14 features in total
    if type(dataset[0][j]) == str:
        labelencoder = LabelEncoder()
        dataset[:, j] = labelencoder.fit_transform(dataset[:, j])

In [24]:
#Test! if data is well transformed
print dataset[0]
print dataset.shape

[62 1 68268 11 9 2 13 0 4 1 0 0 40 38]
(45222, 14)


In [26]:
#split dataset back to train, validation, test
train_X, val_X, test_X = dataset[:27158], dataset[27158:(27158+3004)], dataset[(27158+3004):]
print train_X[0]
print val_X[0]
print test_X[0]
print train_X.shape, val_X.shape, test_X.shape

[62 1 68268 11 9 2 13 0 4 1 0 0 40 38]
[57 2 96779 4 3 2 2 0 4 1 0 0 40 38]
[25 2 226802 1 7 4 6 3 2 1 0 0 40 38]
(27158, 14) (3004, 14) (15060, 14)


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score,KFold,GridSearchCV

clf = RandomForestClassifier(n_estimators=100)
#cv_results = cross_val_score(clf,train_X,train_Y,cv=5)
#print("CV validation accuracy", cv_results.mean()*100)
clf.fit(train_X,train_Y)
prediction = clf.predict(test_X)
print("Accuracy: %f " % (100*accuracy_score(test_Y, prediction)))

Accuracy: 84.840637 


In [29]:
pred_train = clf.predict(train_X)
print("Accuracy: %f " % (100*accuracy_score(train_Y, pred_train)))

Accuracy: 99.992636 


In [30]:
clf=RandomForestClassifier(n_estimators=50,max_features=5,min_samples_leaf=50)
clf.fit(train_X,train_Y)

prediction = clf.predict(test_X)
print("Test dataset: Accuracy: %f " % (100*accuracy_score(test_Y, prediction)))

pred_train = clf.predict(train_X)
print("Train dataset: Accuracy: %f " % (100*accuracy_score(train_Y, pred_train)))

Test dataset: Accuracy: 85.391766 
Train dataset: Accuracy: 86.007806 


In [11]:
print ('feature significances：%s' % clf.feature_importances_)

feature significances：[5.87845124e-02 1.86426851e-03 1.09606442e-01 1.18811449e-01
 3.01342489e-02 4.62363437e-02 1.23995072e-03 5.51060953e-04
 2.26334084e-03 4.44255291e-03 1.63378289e-03 1.30447289e-04
 0.00000000e+00 1.53158052e-03 3.59101731e-03 2.27761550e-05
 5.72364858e-05 7.55650482e-04 1.46172631e-03 1.07144674e-03
 2.52291228e-04 4.35839639e-04 1.69342674e-02 7.49609652e-03
 1.02276980e-02 2.49053563e-02 0.00000000e+00 1.20339537e-02
 2.45263080e-03 1.08382657e-02 0.00000000e+00 1.38508807e-01
 1.66538948e-04 5.42042666e-02 9.95776670e-04 9.38145887e-04
 4.88198419e-03 0.00000000e+00 4.09223432e-03 1.77352471e-02
 4.27566942e-03 2.48110931e-03 4.00696211e-03 1.18504346e-02
 2.04256778e-04 2.63433486e-02 4.18868104e-04 4.87516473e-04
 1.36254412e-03 1.22033701e-03 9.22762250e-02 2.38172648e-02
 2.23779125e-03 2.06785318e-02 1.40466153e-02 1.39011716e-02
 2.27799819e-04 2.18980879e-04 1.91134566e-03 0.00000000e+00
 2.20405061e-03 4.01940970e-02 2.08345608e-02 0.00000000e+00
 0